# **數字手寫影像資料集 Keras Mnist Dataset**


* 資料集來自Keras中數字手寫資料集
* 輸入特徵維度大小是28×28
* 訓練集60000張，測試集10000張



# **02. 讀取資料**
* Keras中的數字手寫資料集 [mnist.load_data()](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data)，回傳訓練與測試集的影像與標籤
* Collections中的Counter，可以計算每一元素的數量


In [ ]:
from tensorflow.keras.datasets import mnist
from collections import Counter 

(x_train_image, y_train_label),(x_test_image, y_test_label) = mnist.load_data() 

print("訓練集的類別數量:%s" %Counter(y_train_label))
print("測試集的類別數量:%s" %Counter(y_test_label))
print("訓練集的維度:",x_train_image.shape)
print("測試集的維度:",x_test_image.shape)

# **03. 繪製影像**
## A. 繪製多張影像
* Matplotlib中的 `subplot(y, x, n)` [[連結]](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.subplot.html)，可以放置子影像y為列數、x為行數與第n個子影像
* Matplotlib中的 `set_title(str)` [[連結]](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.set_title.html)，可以放置子影像的標題
* Matplotlib繪製與儲存影像時，預設為RGB彩色通道，若要以灰階方式繪製與儲存時，可以給予`cmap="gray"`或`plt.cm.gray`

In [ ]:
import matplotlib.pyplot as plt

# 設置影像大小
plt.figure(figsize=(15, 10))

# for迴圈放置子影像
for i in range(15):
  # 設置子影像的行列數與第n個子影像
  ax = plt.subplot(3,5, 1+i)
  # 給予顯示子影像，並灰階顯示
  ax.imshow(x_train_image[i], cmap="gray")
  # 給予子影像的標題 
  ax.set_title("Label: %s" %(y_train_label[i]), size=20)
  # 儲存影像
  plt.imsave(str(y_train_label[i])+".png",x_train_image[i], cmap = plt.cm.gray)
# 顯示影像
plt.show()

# **資料前處理**
## A. 獨熱編碼
* keras中的 `to_categorical(label)` [[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical)，將`label`作獨熱編碼（One-hot Encoding）

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

# 將排序型編碼的標籤另存變數，於後算混淆矩陣時會使用到
sort_train_label = y_train_label
sort_test_label = y_test_label

# 將排序型編碼轉為獨熱編碼
y_train_label = to_categorical(y_train_label)
y_test_label = to_categorical(y_test_label)

## B. 正規化


In [ ]:
# 正規化與轉換型別
x_train_image = x_train_image.astype('float32')/255
x_test_image = x_test_image.astype('float32')/255

## C. 改變維度
* Numpy中的 `expand_dims(array, axis=num)` [[連結]](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html)，可將陣列array在選擇的軸維度中增加⼀維度

In [ ]:
# 打印維度
print("轉換前的維度:", x_train_image.shape)

# 增加維度
x_train_image = np.expand_dims(x_train_image,-1)
x_test_image = np.expand_dims(x_test_image,-1)

# 打印維度
print("轉換後的維度:", x_train_image.shape)

# **05. 模型建立**
## A. 建立模型

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# 建立順序型模型
cnn = Sequential()

# 增加卷積層，並設置濾波器數量及大小、激勵函數、零填充
cnn.add(Conv2D(32,(4,4),activation='relu', padding='same', input_shape=(28,28,1)))
# 增加捨棄層
cnn.add(Dropout(0.2))
# 增加最大池化層
cnn.add(MaxPooling2D((2,2)))

cnn.add(Conv2D(64,(3,3),activation='relu', padding='same'))
cnn.add(Dropout(0.2))

cnn.add(Conv2D(128,(2,2),activation='relu', padding='same'))
cnn.add(Dropout(0.2))

# 拉成一維度
cnn.add(Flatten())
# 增加全連接層
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(10, activation='softmax'))

cnn.summary()

## B. 編譯方式
* Keras中compile之損失，我們選擇 `categorical_crossentropy` [[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/categorical_crossentropy)
* Keras中compile之最佳化梯度下降法，我們選擇 `Adam` [[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)
* Keras中compile之Metrics，可以自定義評價函數，給予字串名稱

In [ ]:
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

## C. 訓練模型
* Keras中的 `history = model.fit(x, y, batch_size, epochs, validation_data, validation_split, verbose)`[[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/Model)
  * x, y為訓練集的輸入特徵與標籤
  * batch_size為批量數量，為多少資料計算一次損失值
  * epochs為迭代次數
  * validation_data可將驗證集在訓練過程中，記錄每次的迭代驗證指標值，此資料並不會參與訓練過程，預設為None，若以設validation_split則無法使用此參數
  * validation_split可將訓練集分割一部分作為驗證集，來記錄每次的迭代驗證指標值，預設為0.0，若以設validation_data則無法使用此參數
  * verbose為訓練的詳細程度，預設為1。靜音為0，進度條為1，詳細模式為2
  * history為訓練過程中回返的詳細數值，若不清楚記錄的變數為何，可以打印`history.history.keys()`查看key

* 補充：如果嫌繪製圖表麻煩，可以使用TensorBoard來快速紀錄訓練過程。
  * `from tensorflow.keras.callbacks import TensorBoard`
  * `tensorboard = TensorBoard(log_dir="my_log")`
  * `history = model.fit(..., callbacks=[tensorboard])`
  * 命令提示字元用 `cd` 指令至程式碼的位置，並輸入`tensorboard --logdir=my_log`，會跳出一網址連結，輸入至瀏覽器即可開啟TensorBoard




In [ ]:
history = cnn.fit(x=x_train_image, y=y_train_label, batch_size=200, epochs=10, validation_data=(x_test_image, y_test_label), verbose=1)

# **06. 驗證指標**
## A. 繪製學習曲線：損失函數

In [ ]:
acc = history.history['acc']
test_acc = history.history['val_acc']
loss = history.history['loss']
test_loss = history.history['val_loss']

plt.rcParams["font.family"] = "serif"
plt.title("Training & Test", fontsize=20)
plt.xlabel("Iteration", fontsize=18)
plt.ylabel("Accuracy", fontsize=18)
plt.plot(np.arange(len(acc)), acc,color='b', label="Training set", marker='o', markersize=5)
plt.plot(np.arange(len(test_acc)), test_acc,color='r', label="Test set", marker='o', markersize=5)
plt.xticks(np.linspace(0,9,10),fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right',fontsize=14)
plt.show()

plt.title("Training & Test", fontsize=20)
plt.xlabel("Iteration", fontsize=18)
plt.ylabel("Loss", fontsize=18)
plt.plot(np.arange(len(loss)), loss,color='b', label="Training set", marker='o', markersize=5)
plt.plot(np.arange(len(test_loss)), test_loss,color='r', label="Test set", marker='o', markersize=5)
plt.xticks(np.linspace(0,9,10),fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right',fontsize=14)
plt.show()

## B. 損失值與準確度
* Keras中的model.evaluate [[連結]](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate)，可以進行批量的計算損失與準確度

In [ ]:
train_loss, train_acc = cnn.evaluate(x_train_image, y_train_label)
print("訓練集的準確度為：%0.4f" %(train_acc))
print("訓練集的損失值為：%0.4f" %(train_loss))

test_loss, test_acc = cnn.evaluate(x_test_image, y_test_label)
print("測試集的準確度為：%0.4f" %(test_acc))
print("測試集的損失值為：%0.4f" %(test_loss))


## C. 訓練集的混淆矩陣

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

predict = cnn.predict(x_train_image)
predictions = [np.argmax(one_hot)for one_hot in predict]

cm = confusion_matrix(sort_train_label, predictions)
plt.figure(figsize=(10,10))
plt.title('Training set - Accuracy: %0.4f' %(train_acc), size = 20)
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=1.0, square = True, cmap = 'Blues',annot_kws={"size": 16})
plt.ylabel('Actual label', size = 18)
plt.xlabel('Predicted label', size = 18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

## D. 測試集的混淆矩陣

In [ ]:
predict = cnn.predict(x_test_image)
predictions = [np.argmax(one_hot)for one_hot in predict]

cm = confusion_matrix(sort_test_label, predictions)
plt.figure(figsize=(10,10))
plt.title('Test set - Accuracy: %0.4f' %(test_acc), size = 20)
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=1.0, square = True, cmap = 'Oranges',annot_kws={"size": 16})
plt.ylabel('Actual label', size = 18)
plt.xlabel('Predicted label', size = 18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

# **07. 儲存與讀取模型**
## A. 模型存取

In [ ]:
cnn.save('cnn_mnist.h5')

from tensorflow.keras.models import load_model
cnn = load_model('cnn_mnist.h5')

# B. 單筆影像測試

In [ ]:
import cv2
img = cv2.imread("9.png")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(gray_img, cmap="gray")
plt.show()

resize_img = np.resize(gray_img, (28, 28))
test_image = resize_img.astype('float32')/255
test_image = np.expand_dims(test_image,-1)
test_image = np.expand_dims(test_image,0)
print(test_image.shape)

predict = cnn.predict(test_image)
print(predict)
predictions = [np.argmax(one_hot)for one_hot in predict]
print(predictions)